In [76]:
import pandas as pd
import numpy as np

In [92]:
purchase_data = pd.read_excel('chapter-12-relay-foods.xlsx', sheetname=1)
purchase_data.head()

,OrderId,OrderDate,UserId,TotalCharges,CommonId,PupId,PickupDate
0,262,2009-01-11,47,50.67,TRQKD,2,2009-01-12
1,278,2009-01-20,47,26.60,4HH2S,3,2009-01-20
2,294,2009-02-03,47,38.71,3TRDC,2,2009-02-04
3,301,2009-02-06,47,53.38,NGAZJ,2,2009-02-09
4,302,2009-02-06,47,14.28,FFYHD,2,2009-02-09


In [93]:
purchase_data['OrderPeriod'] = purchase_data['OrderDate'].dt.to_period('M')

In [95]:
len(purchase_data)

2891

In [33]:
purchase_data.head()

,OrderId,OrderDate,UserId,TotalCharges,CommonId,PupId,PickupDate,OrderPeriod
0,262,2009-01-11,47,50.67,TRQKD,2,2009-01-12,2009-01
1,278,2009-01-20,47,26.60,4HH2S,3,2009-01-20,2009-01
2,294,2009-02-03,47,38.71,3TRDC,2,2009-02-04,2009-02
3,301,2009-02-06,47,53.38,NGAZJ,2,2009-02-09,2009-02
4,302,2009-02-06,47,14.28,FFYHD,2,2009-02-09,2009-02


In [98]:
month_cohort = purchase_data[['UserId', 'OrderPeriod']].groupby('UserId').min()
month_cohort.rename(columns={'OrderPeriod': 'UserCohort'}, inplace=True)
month_cohort.head()

,UserCohort
UserId,
47,2009-01
95,2009-03
98,2009-01
112,2009-01
141,2009-11


In [102]:
month_cohort = month_cohort.reset_index()
month_cohort.head()

,UserId,UserCohort
0,47,2009-01
1,95,2009-03
2,98,2009-01
3,112,2009-01
4,141,2009-11


In [103]:

purchase_data = purchase_data.merge(month_cohort)
len(purchase_data)

2891

In [104]:
purchase_data.head()

,OrderId,OrderDate,UserId,TotalCharges,CommonId,PupId,PickupDate,OrderPeriod,UserCohort
0,262,2009-01-11,47,50.67,TRQKD,2,2009-01-12,2009-01,2009-01
1,278,2009-01-20,47,26.60,4HH2S,3,2009-01-20,2009-01,2009-01
2,294,2009-02-03,47,38.71,3TRDC,2,2009-02-04,2009-02,2009-01
3,301,2009-02-06,47,53.38,NGAZJ,2,2009-02-09,2009-02,2009-01
4,302,2009-02-06,47,14.28,FFYHD,2,2009-02-09,2009-02,2009-01


Calculate the sum of all orders in each cohort

In [106]:
cohort_data = purchase_data.groupby(['UserCohort', 'OrderPeriod']).agg({'UserId': pd.Series.nunique,
                                                                        'OrderId': pd.Series.nunique,
                                                                        'TotalCharges': np.sum})
cohort_data.rename(columns={'UserId': 'TotalUsers', 'OrderId': 'TotalOrders'}, inplace=True)
cohort_data.head()

TotalUsers  TotalCharges  TotalOrders
UserCohort OrderPeriod                                       
2009-01    2009-01              22      1850.255           30
           2009-02               8      1351.065           25
           2009-03              10      1357.360           26
           2009-04               9      1604.500           28
           2009-05              10      1575.625           26

For next class, create pivot table with cohort period in vertical index, UserCohort as horizontal index, and total active users during the intersection of the two for the entries.

In [119]:
clone = cohort_data

cohort_size = clone[['UserCohort', 'TotalUsers']].groupby(['UserCohort']).max()

In [121]:
def calc_retention(row):
    return row.TotalUsers / cohort_size.loc[row.UserCohort]

clone['ActivePercentage'] = clone.apply(calc_retention, axis=1)
clone.head()

,level_0,index,UserCohort,OrderPeriod,TotalUsers,TotalCharges,TotalOrders,ActivePercentage
0,0,0,2009-01,2009-01,22,1850.255,30,1.000000
1,1,1,2009-01,2009-02,8,1351.065,25,0.363636
2,2,2,2009-01,2009-03,10,1357.360,26,0.454545
3,3,3,2009-01,2009-04,9,1604.500,28,0.409091
4,4,4,2009-01,2009-05,10,1575.625,26,0.454545


In [ ]:
cohort_data = clone[['UserCohort', 'OrderPeriod', 'TotalUsers', 'TotalCharges', 'TotalOrders', 'ActivePercentage']]
cohort_data.